In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import svgwrite
import cairosvg
from scipy.interpolate import CubicSpline
import cv2
# Read Polyline Data
def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

# Visualize the Polyline Data
def plot(paths_XYs, colours=['r', 'g', 'b', 'c', 'm', 'y', 'k']):
    fig, ax = plt.subplots(tight_layout=True, figsize=(8, 8))
    for i, XYs in enumerate(paths_XYs):
        c = colours[i % len(colours)]
        for XY in XYs:
            if len(XY.shape) == 1:
                # Reshape if XY is a 1D array
                XY = XY.reshape(-1, 2)
            if XY.shape[1] == 2:
                ax.plot(XY[:, 0], XY[:, 1], c=c, linewidth=2)
            else:
                print(f"Unexpected shape for polyline: {XY.shape}")
    ax.set_aspect('equal')
    plt.show()
# Example usage
paths_XYs = read_csv('frag0.csv')
plot(paths_XYs)

paths_XYs = read_csv('frag01_sol.csv')
plot(paths_XYs)

paths_XYs = read_csv('frag1.csv')
plot(paths_XYs)

paths_XYs = read_csv('frag2.csv')
plot(paths_XYs)

paths_XYs = read_csv('frag2_sol.csv')
plot(paths_XYs)

paths_XYs = read_csv('isolated.csv')
plot(paths_XYs)

paths_XYs = read_csv('isolated_sol.csv')
plot(paths_XYs)

paths_XYs = read_csv('occlusion1.csv')
plot(paths_XYs)

paths_XYs = read_csv('occlusion1_sol.csv')
plot(paths_XYs)

paths_XYs = read_csv('occlusion2_sol.csv')
plot(paths_XYs)

# Identifying and Regularizing Curves
def regularize_curves(paths_XYs, max_iterations=10000):
    regular_shapes = []
    for path in paths_XYs:
        for polyline in path:
            if is_straight_line(polyline):
                regular_shapes.append(('line', polyline))
            elif is_circle(polyline):
                regular_shapes.append(('circle', polyline))
            elif is_rectangle(polyline):
                regular_shapes.append(('rectangle', polyline))
            elif is_rounded_rectangle(polyline):
                regular_shapes.append(('rounded_rectangle', polyline))
            elif is_regular_polygon(polyline):
                regular_shapes.append(('polygon', polyline))
            elif is_star_shape(polyline):
                regular_shapes.append(('star', polyline))
            else:
                regular_shapes.append(('unknown', polyline))
    return regular_shapes

def is_straight_line(polyline, max_iterations=10000):
    if len(polyline) < 2:
        return False
    x0, y0 = polyline[0]
    x1, y1 = polyline[1]
    initial_slope = float('inf') if x1 == x0 else (y1 - y0) / (x1 - x0)
    for i in range(1, len(polyline) - 1):
        x0, y0 = polyline[i]
        x1, y1 = polyline[i + 1]
        slope = float('inf') if x1 == x0 else (y1 - y0) / (x1 - x0)
        if slope != initial_slope:
            return False
    return True

def is_circle(polyline, tolerance=0.1):
    if len(polyline) < 3:
        return False
    centroid = np.mean(polyline, axis=0)
    distances = np.linalg.norm(polyline - centroid, axis=1)
    mean_distance = np.mean(distances)
    return np.all(np.abs(distances - mean_distance) < tolerance)

def is_rectangle(polyline, tolerance=0.1):
    if len(polyline) != 4:
        return False
    def distance(p1, p2):
        return np.linalg.norm(p1 - p2)
    def angle(p1, p2, p3):
        v1 = p1 - p2
        v2 = p3 - p2
        cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        return np.arccos(cos_theta)
    d = [distance(polyline[i], polyline[(i + 1) % 4]) for i in range(4)]
    if not (np.abs(d[0] - d[2]) < tolerance and np.abs(d[1] - d[3]) < tolerance):
        return False
    for i in range(4):
        theta = angle(polyline[i], polyline[(i + 1) % 4], polyline[(i + 2) % 4])
        if not np.abs(theta - np.pi / 2) < tolerance:
            return False
    return True

def is_rounded_rectangle(polyline, tolerance=0.1):
    if len(polyline) < 8:
        return False
    def distance(p1, p2):
        return np.linalg.norm(p1 - p2)
    def is_straight_segment(points):
        if len(points) < 2:
            return False
        p0 = points[0]
        p1 = points[-1]
        for p in points[1:-1]:
            if not np.abs(np.cross(p1 - p0, p - p0)) < tolerance:
                return False
        return True
    def is_rounded_corner(points):
        if len(points) < 4:
            return False
        center = np.mean(points, axis=0)
        distances = np.linalg.norm(points - center, axis=1)
        mean_distance = np.mean(distances)
        return np.all(np.abs(distances - mean_distance) < tolerance)
    n = len(polyline)
    segment_length = (n-4) // 4
    if not np.allclose(polyline[0], polyline[-1], atol=tolerance):
        return False
    straight_segments = [
        polyline[i*segment_length : (i+1)*segment_length] for i in range(4)
    ]
    rounded_corners = [
        polyline[(i+1)*segment_length : (i+2)*segment_length] for i in range(4)
    ]
    for segment in straight_segments:
        if not is_straight_segment(segment):
            return False
    for corner in rounded_corners:
        if not is_rounded_corner(corner):
            return False
    return True

def is_regular_polygon(polyline, tolerance=0.1):
    if len(polyline) < 3:
        return False
    def distance(p1, p2):
        return np.linalg.norm(p1 - p2)
    def angle(p1, p2, p3):
        v1 = p1 - p2
        v2 = p3 - p2
        cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        return np.arccos(np.clip(cos_theta, -1.0, 1.0))
    n = len(polyline)
    if n < 3 or n % 2 != 0:
        return False
    distances = [distance(polyline[i], polyline[(i + 1) % n]) for i in range(n)]
    mean_distance = np.mean(distances)
    if not np.all(np.abs(distances - mean_distance) < tolerance):
        return False
    angles = [angle(polyline[i], polyline[(i + 1) % n], polyline[(i + 2) % n]) for i in range(n)]
    mean_angle = np.mean(angles)
    if not np.all(np.abs(angles - mean_angle) < tolerance):
        return False
    return True

def is_star_shape(polyline, tolerance=0.1):
    if len(polyline) < 10:
        return False
    def distance(p1, p2):
        return np.linalg.norm(p1 - p2)
    def angle(p1, p2, p3):
        v1 = p1 - p2
        v2 = p3 - p2
        cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        return np.arccos(np.clip(cos_theta, -1.0, 1.0))
    def check_star_pattern(points):
        n = len(points)
        if n % 2 != 0:
            return False
        outer_points = points[0::2]
        inner_points = points[1::2]
        if len(outer_points) != len(inner_points):
            return False
        outer_distances = [distance(outer_points[i], outer_points[(i + 1) % len(outer_points)]) for i in range(len(outer_points))]
        inner_distances = [distance(inner_points[i], inner_points[(i + 1) % len(inner_points)]) for i in range(len(inner_points))]
        mean_outer_distance = np.mean(outer_distances)
        mean_inner_distance = np.mean(inner_distances)
        if not (np.all(np.abs(outer_distances - mean_outer_distance) < tolerance) and
                np.all(np.abs(inner_distances - mean_inner_distance) < tolerance)):
            return False
        for i in range(len(outer_points)):
            p1 = outer_points[i]
            p2 = outer_points[(i + 1) % len(outer_points)]
            p3 = inner_points[i]
            p4 = inner_points[(i + 1) % len(inner_points)]
            angle1 = angle(p1, p2, p3)
            angle2 = angle(p3, p4, p1)
            if not (np.abs(angle1 - angle2) < tolerance):
                return False
        return True
    return check_star_pattern(polyline)

# Checking for Symmetry
def has_reflection_symmetry(polyline, tolerance=0.1):
    contour = polyline.astype(np.int32)
    moments = cv2.moments(contour)
    # Calculate central moments
    mu20 = moments['mu20']
    mu02 = moments['mu02']
    mu11 = moments['mu11']

    # Calculate the angle of the principal axis
    theta = 0.5 * np.arctan2(2 * mu11, mu20 - mu02)

    # Rotate the polyline to align with the principal axis
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                               [np.sin(theta), np.cos(theta)]])
    rotated_polyline = np.dot(polyline - np.mean(polyline, axis=0), rotation_matrix) + np.mean(polyline, axis=0)

    # Split the rotated polyline into two halves, handling odd number of points
    midpoint = len(rotated_polyline) // 2
    half1 = rotated_polyline[:midpoint]
    half2 = rotated_polyline[midpoint:]
    # If odd number of points, discard the middle point to ensure equal halves
    if len(rotated_polyline) % 2 == 1:
        half2 = half2[1:]

    # Reflect one half and compare to the other
    reflected_half1 = np.copy(half1)
    reflected_half1[:, 0] = -reflected_half1[:, 0]  # Reflect across the y-axis
    return np.allclose(reflected_half1, half2, atol=tolerance)
    def is_mirrored_segment(segment, axis):
        mirrored_segment = np.copy(segment)
        mirrored_segment[:, axis] = -mirrored_segment[:, axis]
        return np.allclose(segment, mirrored_segment, atol=tolerance)
    def mirror_polyline(polyline, axis):
        mirrored_polyline = np.copy(polyline)
        mirrored_polyline[:, axis] = -mirrored_polyline[:, axis]
        return mirrored_polyline
    mirrored_x = mirror_polyline(polyline, 0)
    mirrored_y = mirror_polyline(polyline, 1)
    return np.allclose(polyline, mirrored_x, atol=tolerance) or np.allclose(polyline, mirrored_y, atol=tolerance)

def has_rotational_symmetry(polyline, tolerance=0.1):
    def rotate_polyline(polyline, angle, origin):
        rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
        rotated_polyline = np.dot(polyline - origin, rotation_matrix) + origin
        return rotated_polyline
    def check_rotational_symmetry(polyline, origin, angles):
        for angle in angles:
            rotated_polyline = rotate_polyline(polyline, angle, origin)
            if np.allclose(polyline, rotated_polyline, atol=tolerance):
                return True
        return False
    centroid = np.mean(polyline, axis=0)
    return check_rotational_symmetry(polyline, centroid, [np.pi/2, np.pi, 3*np.pi/2])

# Completing the Curve
def complete_curves(path_XYs):
    completed_curves = []
    for path in path_XYs:
        for polyline in path:
            if has_gaps(polyline):
                completed_polyline = complete_polyline(polyline)
                completed_curves.append(completed_polyline)
            else:
                completed_curves.append(polyline)
    return completed_curves

def has_gaps(polyline, threshold=5.0):
    distances = np.linalg.norm(np.diff(polyline, axis=0), axis=1)
    return np.any(distances > threshold)

def complete_polyline(polyline):
    if len(polyline) < 2:
        return polyline
    gaps = identify_gaps(polyline)
    for start, end in gaps:
        completed_segment = interpolate_segment(polyline[start], polyline[end])
        polyline = np.insert(polyline, start + 1, completed_segment, axis=0)
    return polyline

def identify_gaps(polyline, threshold=5.0):
    distances = np.linalg.norm(np.diff(polyline, axis=0), axis=1)
    gaps = np.where(distances > threshold)[0]
    return [(gap, gap + 1) for gap in gaps]

def interpolate_segment(start, end, num_points=50):
    t = np.linspace(0, 1, num_points)
    cs = CubicSpline([0, 1], np.vstack([start, end]), axis=0)
    return cs(t)

# Generating SVG from Polylines

import svgwrite

def polylines_to_svg(polylines, svg_filename, width=500, height=500):
    dwg = svgwrite.Drawing(svg_filename, size=(width, height))
    for polyline in polylines:
        points = [(float(x), float(y)) for x, y in polyline]
        dwg.add(dwg.polyline(points, stroke=svgwrite.rgb(10, 10, 16, '%'), fill='none', stroke_width=2))
    dwg.save()

# Rasterizing SVG
def rasterize_svg(svg_filename, output_filename='output.png'):
    cairosvg.svg2png(url=svg_filename, write_to=output_filename)

# Evaluation Functionality
from scipy.spatial.distance import cdist

def evaluate_results(ground_truth, predicted, tolerance=1e-2):
    # Flatten and reshape data
    try:
        flat_ground_truth = np.concatenate([np.reshape(arr, (-1, 2)) for path in ground_truth for arr in path])
        flat_predicted = np.concatenate([np.reshape(arr, (-1, 2)) for arr in predicted])
    except Exception as e:
        print("Error in flattening data:", e)
        return 0.0

    # Check if data lengths match
    if len(flat_ground_truth) == 0 or len(flat_predicted) == 0:
        return 0.0

    # Calculate distance between each pair of points in ground truth and predicted
    distances = cdist(flat_ground_truth, flat_predicted)

    # Count matches within tolerance
    matches = np.sum(np.min(distances, axis=1) < tolerance)

    return matches / len(flat_ground_truth) if len(flat_ground_truth) > 0 else 0.0

# Example Usage
if __name__ == '__main__':
    path_XYs = read_csv('frag2.csv')
    sol_XYs = read_csv('frag2_sol.csv')
    plot(path_XYs)

    regular_shapes = regularize_curves(path_XYs)
    print("Detected shapes and regularized curves:")
    for shape, polyline in regular_shapes:
        print(f"{shape}: {polyline[:5]}...")  # Print the first 5 points as a preview

    has_symmetry = any(has_reflection_symmetry(polyline) or has_rotational_symmetry(polyline) for _, polyline in regular_shapes)
    print(f"Detected symmetry: {has_symmetry}")

    completed_curves = complete_curves(path_XYs)
    plot(completed_curves)

    polylines_to_svg(completed_curves, 'completed.svg')
    rasterize_svg('completed.svg', 'completed.png')

    evaluation_score = evaluate_results(sol_XYs, completed_curves)
    print(f"Evaluation score: {evaluation_score}")
